# 1. Подготовка

In [1]:
#импорт необходимых библиотек
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score, make_scorer
from lightgbm import LGBMClassifier
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer 
import re
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
#выгрузка данных
data_full = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
data_act = data_full.sample(80000).reset_index(drop = True)

In [3]:
#проверка данных
display(data_full.head(10))
data_full.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
#выделение регулярных выражений
def clear_text(data):
    data_red = data.copy()
    for i in range(len(data)):
        text = re.sub(r'[^a-zA-Z]',' ', data.loc[i, 'text'])
        text = text.split()
        text = " ".join(text)
        data_red.loc[i, 'clr_text'] = text
        if i % 10000 == 0:
            print('Урааааа, новые 10-тысяч!', i)
    return data_red

In [5]:
#функция леммитизации
def lemmatize(data):
    m = WordNetLemmatizer()
    for i in range(len(data)):
        data.loc[i,'lemm_text'] = ''.join(m.lemmatize(data.loc[i, 'clr_text']))
        if i % 10000 == 0:
            print('Урааааа, новые 10-тысяч!', i)
        
    return data

In [6]:
#чистим данные от регулярных выражений
data_clr = clear_text(data_act)

Урааааа, новые 10-тысяч! 0
Урааааа, новые 10-тысяч! 10000
Урааааа, новые 10-тысяч! 20000
Урааааа, новые 10-тысяч! 30000
Урааааа, новые 10-тысяч! 40000
Урааааа, новые 10-тысяч! 50000
Урааааа, новые 10-тысяч! 60000
Урааааа, новые 10-тысяч! 70000


In [8]:
#леммитизация данных
data_lem = lemmatize(data_clr)

Урааааа, новые 10-тысяч! 0
Урааааа, новые 10-тысяч! 10000
Урааааа, новые 10-тысяч! 20000
Урааааа, новые 10-тысяч! 30000
Урааааа, новые 10-тысяч! 40000
Урааааа, новые 10-тысяч! 50000
Урааааа, новые 10-тысяч! 60000
Урааааа, новые 10-тысяч! 70000


In [24]:
display('Текс без леммитизации:', data_act['text'][0])
display('Текс после леммитизации:',data_lem['lemm_text'][0])

'Текс без леммитизации:'

"I'M NOT BLOCKED BEACAUSE I DID NOT DO ANYTHING BUT MAKE A PAGE"

'Текс после леммитизации:'

'I M NOT BLOCKED BEACAUSE I DID NOT DO ANYTHING BUT MAKE A PAGE'

In [10]:
features = data_lem.drop('toxic', axis = 1)
target = data_lem['toxic']

In [11]:
#делим данные на тестовую и обучающую выборку
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

In [12]:
#находим TF-IDF для данных
corpus_train = features_train['lemm_text'].values.astype('U')
corpus_test = features_test['lemm_text'].values.astype('U')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train) 
tf_idf_test = count_tf_idf.transform(corpus_test) 
print("Размер матрицы обучающих данных:", tf_idf_train.shape)
print("Размер матрицы тестовых данных:", tf_idf_test.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер матрицы обучающих данных: (60000, 96138)
Размер матрицы тестовых данных: (20000, 96138)


In [25]:
#определение гипепараметров для модели "случайный лес"
#model_for = RandomForestClassifier()

#parametrs = {'max_depth': range(1,6), 'n_estimators': range(1, 21, 10)}
#grid_search = GridSearchCV(model_for, parametrs, scoring = f1)
#grid_search.fit(tf_idf_train, target_train)
#grid_search.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: Und

{'max_depth': 5, 'n_estimators': 1}

In [15]:
#определение гипепараметров для модели "дерево решений"
#model_tree = DecisionTreeClassifier()

#parametrs = {'max_depth': range(1,6), 'min_samples_split': range(2, 6, 2)}
#grid_search = GridSearchCV(model_tree, parametrs, scoring = f1)
#grid_search.fit(tf_idf_train, target_train)
#grid_search.best_params_

In [16]:
#определение гипепараметров для модели "логистичская регерссия"
#model_log = LogisticRegression()

#parametrs = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
#grid_search = GridSearchCV(model_log, parametrs, scoring = f1)
#grid_search.fit(tf_idf_train, target_train)
#grid_search.best_params_

In [17]:
#определение гипепараметров для модели "LGB"
#model_lgb = LGBMClassifier(boosting_type ='gbdt')

#parametrs = {'max_depth': range(1,3), 'num_leaves': range(31, 93, 31)}
#grid_search = GridSearchCV(model_lgb, parametrs, scoring = f1)
#grid_search.fit(tf_idf_train, target_train)
#grid_search.best_params_

### Выводы. Шаг 1
1) Для удобства и быстроты обучения модели, беру не весь набор данных, а только 80к записей.    
2) Проведена леммитизация предложений и проведен анализ регулярных выражений.   
3) Найдены гиперпараметры для каждой модели

# 2. Обучение

In [18]:
#обучение модели "Случайный лес"
model_for = RandomForestClassifier(max_depth=5, n_estimators=1)

#предсказание модели "Случайный лес"
model_for.fit(tf_idf_train, target_train)
predict_for = model_for.predict(tf_idf_test)
print('Значение F1 для случайного леса (валидационные данные)', "%.2f" % f1_score(predict_for, target_test))

Значение F1 для случайного леса (валидационные данные) 0.02


In [19]:
#обучение модели "Дерево решений"
model_tree = DecisionTreeClassifier(max_depth=5, min_samples_split=4)

#предсказание модели "Дерево решений"
model_tree.fit(tf_idf_train, target_train)
predict_tree = model_tree.predict(tf_idf_test)
print('Значение F1 для случайного леса (валидационные данные)', "%.2f" % f1_score(predict_tree, target_test))

Значение F1 для случайного леса (валидационные данные) 0.52


In [20]:
#обучение модели "Логистическая регрессия"
model_log = LogisticRegression(C=100)

#предсказания модели "Логистическая регрессия"
model_log.fit(tf_idf_train, target_train)
predict_log = model_log.predict(tf_idf_test)
print('Значение F1 для логистической регрессии (валидационные данные)', "%.2f" % f1_score(predict_log, target_test))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Значение F1 для логистической регрессии (валидационные данные) 0.77


In [21]:
#обучение модели "LGB"
model_lgb = LGBMClassifier(boosting_type ='gbdt', max_depth=2, num_leaves=31)

#предсказания модели "LGB"
model_lgb.fit(tf_idf_train, target_train)
predict_lgb = model_lgb.predict(tf_idf_test)
print('Значение F1 для LGB (валидационные данные)', "%.2f" % f1_score(predict_lgb, target_test))

Значение F1 для LGB (валидационные данные) 0.54


In [22]:
#таблица с общими результатами
ml = ['Случайный лес', 'Дерево решений', 'Логистическая регрессия', 'LGB']
data = {
  'Результаты RMSE на валидационных данных':  [f1_score(predict_for, target_test), f1_score(predict_tree, target_test), f1_score(predict_log, target_test), f1_score(predict_lgb, target_test)]
}
data_ml = pd.DataFrame(data=data, index=ml)
display(data_ml)

,Результаты RMSE на валидационных данных
Случайный лес,0.015686
Дерево решений,0.520086
Логистическая регрессия,0.767423
LGB,0.541503


# 3. Выводы

Всех лучше себя показала модель ML - логистическая регрессия! На половине данных результат F1 получился 0,76!  

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны